In [1]:
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt
import plotly.io as pio
#import squarify #treemap
import os
import matplotlib
import warnings



#to enable the inline plotting
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

sns.set_style("darkgrid")

In [2]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import StandardScaler


from scipy.stats import normaltest

from pandas_profiling import ProfileReport

from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.style.palettes import PALETTES, SEQUENCES, color_palette

import lazypredict
from lazypredict.Supervised import LazyClassifier

warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Function for EDA. Using the display() function to have  well-formatted tables. We are mainly using pandas to explore the datasets

def dataset_description(df_target):

    print('This is the Dataset shape: %s\n' % (df_target.shape, ))
    print('Dataset columns: %s\n' % df_target.columns)

    print('\nColumns description:\n')
    display(df_target.info())
    display(df_target.describe())  # describe the dataset

    print('\nNull values:\n')
    display(df_target.isnull().sum())  # Identify null values

#function performing a quick check on df_inspection to have best of pandas functions separated by a line
def quick_check(dataframe):
    print('First 5 rows %s\n')
    print(dataframe.head(2))
    print("=====================================")
    print('Dataframe shape %s\n')
    print(dataframe.shape)
    print("=====================================")
    print('Dataframe describe categorical %s\n')
    print(dataframe.describe(include=['O']))
    print("=====================================")
    print('Dataframe null values %s\n')
    print(dataframe.isnull().sum())
    print("=====================================")
    print('Dataframe value counts %s\n')
    print(dataframe.value_counts())
    print("=====================================")

#stats function
def stats(dataframe):
    print('Dataframe correlation %s\n')
    print(dataframe.corr())
    print("=====================================")
    print('Dataframe covariance %s\n')
    print(dataframe.cov())
    print("=====================================")
    print('Dataframe skew %s\n')
    print(dataframe.skew())
    print("=====================================")
    print('Dataframe kurtosis %s\n')
    print(dataframe.kurt())
    print("=====================================")

#create a function to normalize characters from a dataset's column in Spanish
def normalize_characters(df, column):
    df[column] = df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace('á', 'a')
    df[column] = df[column].str.replace('é', 'e')
    df[column] = df[column].str.replace('í', 'i')
    df[column] = df[column].str.replace('ó', 'o')
    df[column] = df[column].str.replace('ú', 'u')
    df[column] = df[column].str.replace('ñ', 'n')
    df[column] = df[column].str.replace('ü', 'u')
    df[column] = df[column].str.replace('ç', 'c')
    df[column] = df[column].str.replace('(', '')
    df[column] = df[column].str.replace(')', '')
    df[column] = df[column].str.replace('\'', '')
    df[column] = df[column].str.replace('´', '')
    df[column] = df[column].str.replace('`', '')
    df[column] = df[column].str.replace('’', '')
    return df.head(2)

#create function to change detypes in64 to int32 in a df
def change_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

In [4]:
#show all print outputs when using a function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#display all columns
pd.set_option('display.max_columns', None)

In [5]:
#point to the folder where the data is stored for Pedro
os.chdir(r"C:\Users\alfon\Desktop\Máster IE - Business Analytics & Big Data\3rd Term\Sustainability Datathon\Data")

# Loading inspection data
df_inspection = pd.read_csv('inspections.csv')

# Loading network data
df_network = pd.read_csv('network.csv')

# Loading submission data
df_submission = pd.read_csv('sample_submission.csv')

# Loading train data
df_train = pd.read_csv('train_consolidated.csv')

In [6]:
df_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909733 entries, 0 to 909732
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   PipeId     909733 non-null  int64
 1   Incidence  909733 non-null  int64
dtypes: int64(2)
memory usage: 13.9 MB


In [7]:
#I want to know how many different PipeId are in df_submission
df_submission['PipeId'].nunique()

909733

There is just one inspection per each pipe in the period 2021 and 2022

In [8]:
sub_pipeline=pd.merge(df_submission, df_network, on='PipeId', how='left')


In [9]:
sub_pipeline.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 909733 entries, 0 to 909732
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   PipeId               909733 non-null  int64  
 1   Incidence            909733 non-null  int64  
 2   Province             909733 non-null  object 
 3   Town                 909733 non-null  object 
 4   YearBuilt            909733 non-null  int64  
 5   Material             909733 non-null  object 
 6   GasType              909733 non-null  object 
 7   Diameter             909733 non-null  float64
 8   Length               909733 non-null  float64
 9   Pressure             909733 non-null  float64
 10  NumConnections       909733 non-null  int64  
 11  NumConnectionsUnder  909733 non-null  int64  
 12  BoolBridle           909733 non-null  bool   
dtypes: bool(1), float64(3), int64(5), object(4)
memory usage: 91.1+ MB


In [10]:
#get the names of all the columns in df_trainç
df_train.columns

Index(['PipeId', 'InspectionYear', 'MonthsLastRev', 'Incidence', 'Province',
       'Town', 'YearBuilt', 'Diameter', 'Length', 'Pressure', 'NumConnections',
       'NumConnectionsUnder', 'BoolBridle', 'No_Inspections',
       'No_Incidences_Total', 'average_severity_pipe', 'relative_risk',
       'preventive_maintenance_rate', 'Probability_rate',
       'Average_MonthsLastRev', 'pipe_inspected_frequently',
       'Age_pipe_at_inspection', 'aspect', 'Relative_Thickness', 'pipe_area',
       'Total_Connections', 'area_connection', 'incidence_area',
       'connection_bool', 'Severity_high', 'Severity_medium', 'Severity_low',
       'gas_natural', 'Material_Acrylonitrile-Butadiene-Styrene',
       'Material_Copper', 'Material_Fiberglass-Reinforced Plastic',
       'Material_Polyethylene', 'Material_Polypropylene', 'Diameter2',
       'Length2', 'Pressure2', 'Average yearly temperature (°C)',
       'Min. Temperature (°C)', 'Max. Temperature (°C)',
       'Yearly Rainfall (mm)', 'Average y

In [11]:
#df_train= df_train.groupby('PipeId').apply(lambda x: x.sort_values('InspectionYear', ascending=False).iloc[0])

In [12]:
#Converting diameter to int
sub_pipeline['Diameter'] = sub_pipeline['Diameter'].astype('int64')

In [13]:
#divide all values in Diameter by 1000 to convert to meters
sub_pipeline['Diameter'] = sub_pipeline['Diameter'] / 1000

In [14]:
#create a new variable called aspect that is equal to pressure divided by diameter multiplied by length
sub_pipeline['aspect']=(sub_pipeline['Pressure']/sub_pipeline['Diameter'])*sub_pipeline['Length']

In [15]:
#create a new column that divides the diameter by the pressure and name it Relative Thickness
sub_pipeline['Relative_Thickness'] = sub_pipeline['Diameter'] / sub_pipeline['Pressure']

In [16]:
#creating a column named pipe_area that multiplies diameter by lenght by pi

sub_pipeline['pipe_area'] = sub_pipeline['Diameter'] * sub_pipeline['Length'] * 3.1416

In [17]:
#Create a column called Total_Connections that adds NumConnections and NumConnectionsUnder
sub_pipeline['Total_Connections'] = sub_pipeline['NumConnections'] + sub_pipeline['NumConnectionsUnder']

In [18]:
#create a column named area_connection that divides pipe_area by NumConnections
sub_pipeline['area_connection'] = sub_pipeline['Total_Connections']/ sub_pipeline['pipe_area'] 

In [19]:
#create a boolean column named connection_bool that is 1 if NumConnections is greater than 1
sub_pipeline['connection_bool'] = np.where(sub_pipeline['NumConnections'] > 1, 1, 0)

In [20]:
# Converting Boolbride into  boolean variable
def boolbridle(x):
    return 1 if x == 'True' else 0

In [21]:
# Apply function on dataset
sub_pipeline['BoolBridle'] =sub_pipeline['BoolBridle'].apply(lambda x: boolbridle(x))

In [22]:
#hot encode GasType column in the train_copy dataframe subset
sub_pipeline = pd.get_dummies(sub_pipeline, columns=['GasType'], prefix = ['GasType'])

In [23]:
#delete GasType_Gas propano column
sub_pipeline = sub_pipeline.drop(['GasType_Gas propano'], axis=1)

In [24]:
#change name of GasType_Gas natural column to gas_natural
sub_pipeline = sub_pipeline.rename(columns={'GasType_Gas natural': 'gas_natural'})
sub_pipeline.head(1)

,PipeId,Incidence,Province,Town,YearBuilt,Material,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural
0,446859944,0,Valencia,Alginet,2012,PE,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1


In [25]:
sub_pipeline.head(1)

,PipeId,Incidence,Province,Town,YearBuilt,Material,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural
0,446859944,0,Valencia,Alginet,2012,PE,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1


In [26]:
print (sub_pipeline['Material'].unique())

['PE' 'PN' 'AO' 'FD' 'CU' 'ZD' 'FI' 'PA' 'FG']


In [27]:
#Map df_combined Material column to the names of materials
sub_pipeline['Material'] = sub_pipeline['Material'].map({'PE': 'Polyethylene', 'AO': 'Acrylonitrile-Butadiene-Styrene', 'FD': 'Fiberglass-Reinforced Plastic', 
    'FG': 'Fiberglass', 'PN': 'Polypropylene', 'PA': 'Polyamide', 'FO': 'Flexible Polyolefin', 'FI': 'Flexible Polyvinyl Chloride', 'CU': 'Copper', 
    'PV': 'Polyvinylidene Fluoride', 'ZD': 'Zinc-Coated Steel', 'ZA': 'Zinc-Aluminum', 'CP': 'Cast Iron', 'CS': 'Cast Steel', 
    'ZC': 'Zinc-Coated Steel', 'ZM': 'Zinc-Magnesium','ZN': 'Zinc', 'AL': 'Aluminum', 'ZP': 'Zinc-Coated Steel', 'ZF': 'Zinc-Aluminum-Magnesium'})

In [28]:
#Hot enconde Material column in the train_copy dataframe subset
sub_pipeline = pd.get_dummies(sub_pipeline, columns=['Material'])
sub_pipeline = sub_pipeline.drop(['Material_Fiberglass', 'Material_Zinc-Coated Steel', 'Material_Flexible Polyvinyl Chloride', 'Material_Polyamide'], axis=1)
sub_pipeline.head(1)

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene
0,446859944,0,Valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0


In [29]:
sub_pipeline.head(1)

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene
0,446859944,0,Valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0


In [30]:
from sklearn.preprocessing import PolynomialFeatures

In [31]:
#create polinomial features in train_copy for Diameter, Length, Pressure
poly = PolynomialFeatures(2)
sub_pipeline['Diameter2'] = poly.fit_transform(sub_pipeline[['Diameter']])[:,2]
sub_pipeline['Length2'] = poly.fit_transform(sub_pipeline[['Length']])[:,2]
sub_pipeline['Pressure2'] = poly.fit_transform(sub_pipeline[['Pressure']])[:,2]
sub_pipeline.head(1)


,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2
0,446859944,0,Valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0,0.01,1961.25,0.02


In [32]:
#point to the folder where the data is stored
os.chdir(r"C:\Users\alfon\Desktop\Máster IE - Business Analytics & Big Data\3rd Term\Sustainability Datathon\Data\Pedro")

# Loading provincias dataset
provincias = pd.read_excel('promedio_tiempo_provincia_anual.xlsx')

In [33]:
provincias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Province                         38 non-null     object 
 1   Average yearly temperature (°C)  38 non-null     float64
 2   Min. Temperature (°C)            38 non-null     float64
 3   Max. Temperature (°C)            38 non-null     float64
 4   Yearly Rainfall (mm)             38 non-null     int64  
 5   Average year Humidity (%)        38 non-null     float64
 6   Rainy days per year (days)       38 non-null     int64  
 7   Yearly Sun Hours (hours)         38 non-null     float64
dtypes: float64(5), int64(2), object(1)
memory usage: 2.5+ KB


In [34]:
normalize_characters(sub_pipeline, 'Province')
normalize_characters(provincias, 'Province')

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2
0,446859944,0,valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0,0.01,1961.25,0.02
1,428124500,0,valencia,Alginet,2012,0.16,110.72,0.15,2,0,0,103.80,1.07,55.65,2,0.04,1,1,0,0,0,1,0,0.03,12257.81,0.02


,Province,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours)
0,lleida,15.46,1.80,32.90,461,0.59,50,112.40
1,la rioja,12.38,1.70,26.90,633,0.72,87,85.20


In [35]:
#join train_copy and provincias on Province
sub_pipeline = sub_pipeline.merge(provincias, on='Province', how='left')

In [36]:
sub_pipeline

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours)
0,446859944,0,valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0,0.01,1961.25,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20
1,428124500,0,valencia,Alginet,2012,0.16,110.72,0.15,2,0,0,103.80,1.07,55.65,2,0.04,1,1,0,0,0,1,0,0.03,12257.81,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20
2,438428871,0,valencia,Alginet,2012,0.11,13.57,0.15,0,0,0,18.51,0.73,4.69,0,0.00,0,1,0,0,0,1,0,0.01,184.28,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20
3,429034569,0,valencia,Alginet,2012,0.11,327.69,0.15,3,0,0,446.85,0.73,113.24,3,0.03,1,1,0,0,0,1,0,0.01,107380.08,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20
4,411184477,0,valencia,Alginet,2007,0.20,932.76,0.15,0,0,0,699.57,1.33,586.07,0,0.00,0,1,0,0,0,1,0,0.04,870044.95,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,0,valencia,Paterna,2004,0.11,7.66,0.03,0,0,0,1.74,4.40,2.65,0,0.00,0,1,0,0,0,1,0,0.01,58.61,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20
909729,235426673,0,valencia,Paterna,2004,0.11,4.38,0.03,2,0,0,1.00,4.40,1.51,2,1.32,1,1,0,0,0,1,0,0.01,19.21,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20
909730,235426707,0,valencia,Paterna,2004,0.11,2.29,0.03,1,0,0,0.52,4.40,0.79,1,1.26,0,1,0,0,0,1,0,0.01,5.25,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20
909731,190956601,0,valencia,Paterna,2004,0.09,87.81,4.00,0,0,0,3902.62,0.02,24.83,0,0.00,0,1,0,0,0,1,0,0.01,7710.42,16.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20
